**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [ ]:
### Download dependency if needed
#import sys
#!{sys.executable} -m pip install scikit-video

###2. Télécharger ffmpeg pré-compilé ici : https://ffmpeg.zeranoe.com/builds/win64/static/ffmpeg-4.1-win64-static.zip

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath("C:\\Users\\Administrator\\Coding\\DeepLearningTP3\\ffmpeg-4.1-win64-static\\bin\\")
import skvideo.io

import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


In [2]:
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

skvideo.io.vwrite('test.mp4',  np.random.uniform(0,255,size=(100,100,100)))
HTML(display_videos('test.mp4'))

# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The act function takes the current state as input and gives the action the agent will make, based on this current state, as an output. The action returned also takes into account a logic of trade-off between exploration and exploitation when training. The idea behind exploration-exploitation is that there is a dilemma that each agent faces when taking a decision: should the agent take advantage of the learned knowledge about the best action or she he continue to explore actions he might have not explored enough?

This trade-off is controlled by the parameter epsilon. As it is clear from the code, the agent will explore random actions with a probability of epsilon and use his learned knowledge Epsilon equaling 0 corresponds to full exploitation (thus potential bias) while unit epsilon corresponds to full exploration (equivalent to random agent).

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=200 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

Board array is the cheese map. It is a grid on which is encoded what each position in the map contains. Either nothing, or a cheese or poison. When the mouse eats the cheese, obviously, the cell becomes empty (ie, the reward of the state becomes 0)

Position array gives the position of the mouse in the grid environment. It has -1 in the boundaries where the rat can't go, 0 where the rat can go and 1 in the position where the rat is.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass
        
        
    def learned_act(self, s):
        temp = np.random.randint(0, self.n_action, size=1)
        return temp[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        pos = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            move = agent.act(pos, train = False) ##we are in test
            
            pos, reward, game_over = env.act(move)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
    
        # Save as a mp4 each 5 iteration
        if e%5 ==0:
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/15.0. Average score (-6.0)
Win/lose count 11.0/9.0. Average score (-2.0)
Win/lose count 12.0/18.0. Average score (-3.3333333333333335)
Win/lose count 9.5/15.0. Average score (-3.875)
Win/lose count 10.0/14.0. Average score (-3.9)
Win/lose count 9.0/20.0. Average score (-5.083333333333333)
Win/lose count 12.0/14.0. Average score (-4.642857142857143)
Win/lose count 12.0/12.0. Average score (-4.0625)
Win/lose count 8.5/14.0. Average score (-4.222222222222222)
Win/lose count 11.0/19.0. Average score (-4.6)
Win/lose count 13.5/9.0. Average score (-3.772727272727273)
Win/lose count 3.0/15.0. Average score (-4.458333333333333)
Win/lose count 8.5/16.0. Average score (-4.6923076923076925)
Win/lose count 5.5/10.0. Average score (-4.678571428571429)
Win/lose count 8.5/10.0. Average score (-4.466666666666667)
Win/lose count 8.0/10.0. Average score (-4.3125)
Win/lose count 5.5/11.0. Average score (-4.382352941176471)
Win/lose count 7.0/15.0. Average score (-4.583333333333333)
Win

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Partie 1 de la question 5__

On a

\begin{align*}
Q^{\pi}(s,a)&=E_{\pi}[\sum_{0\leq t}\gamma^tr(s_t,a_t)\vert s,a] \\
&= r(s,a) +  E_{\pi}[ \gamma \sum_{0\leq t}\gamma^tr(s_{t+1},a_{t+1})\vert s,a] \\
\end{align*}

En conditionnant par rapport aux paires (state, action) de l'étape t = 1 et on utilisant la définition de Q ci-haut, on obtient

\begin{align*}
Q^{\pi}(s,a)&= r(s,a) + \gamma  \sum_{a'}\sum_{s'}p(s_1=s',a_1=a'|s_0=s,a_0=a)  Q^{\pi}(s',a') \\
&= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ 
\end{align*}

__Partie 2 de la question 5__

On utilise ce qu'on vient de démontrer
\begin{align*}
    Q^*(s,a)
    &=  \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \\ 
    &=  r(s,a) +  \max_{\pi} \sum_{a'} \sum_{s'} p(s_1=s', a_1=a' |s_{0}=s,a_{0}=a) \gamma Q^{\pi}(s',a') \\   
    &=  r(s,a) +   \sum_{a'} \sum_{s'} p(s_1=s'|s_{0}=s,a_{0}=a)  \gamma \max_{\pi} Q^{\pi}(s',\pi(s')) \\ 
    &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')] \\
\end{align*}

__Partie 3 de la question 5__
Dans DQN, on va calculer la Q function a partir de la premiere équation et les observations que nous avons. Nous souhaitons que cette Q function corresponde au mapping / policy optimal, ça veut dire, qu'on souhaite que la valeur soit la plus proche que la Q value optimale, estimé à partir de la deuxieme équation. Du coup, c'est naturel de prendre la policy qui minimise l'écart entre ce qu'on aurait (si on choisit cette policy), et notre estimation de la meilleure Q.
Ceci veut dire qu'on va minimiser une fonction cout comme :

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory)>=self.max_memory:
            del self.memory[0]
        self.memory.append(m)
        
        pass

    def random_access(self):
        return self.memory[np.random.choice(range(len(self.memory)))]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [39]:
from keras.optimizers import Adam,sgd
from keras.layers import Flatten

class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        #print(s ,self.n_state)
        return np.argmax(self.model.predict(s.reshape(1,5,5,self.n_state)))
    
    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        
        for i in range(self.batch_size):
            s, n_s, a, r, game_over  =  self.memory.random_access() 
            input_states[i]=s
            target_q[i] = self.model.predict(s.reshape(1,5,5,self.n_state))[0]
            
            
            if game_over_:
                target_q[i][a] = r
            else:
                target_q[i][a]  = r + self.discount*max(self.model.predict(n_s.reshape(1,5,5,self.n_state))[0])
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(32,activation='relu'))
        model.add(Dense(4,activation='relu'))
     
        #adam=Adam(lr=lr, decay = 0.9)
        mse = 'mean_squared_error'
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()
        

In [15]:
epochs_train = 50
env = Environment(grid_size=size, max_time=T, temperature=0.35)
agent = DQN_FC(size, lr=.0005, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1632      
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 132       
Total params: 1,764
Trainable params: 1,764
Non-trainable params: 0
_________________________________________________________________
Epoch 000/050 | Loss 0.1069 | Win/lose count 4.0/8.0 (-4.0)
Epoch 001/050 | Loss 0.0789 | Win/lose count 5.0/5.0 (0.0)
Epoch 002/050 | Loss 0.0638 | Win/lose count 5.5/9.0 (-3.5)
Epoch 003/050 | Loss 0.0698 | Win/lose count 4.0/5.0 (-1.0)
Epoch 004/050 | Loss 0.0805 | Win/lose count 3.5/1.0 (2.5)
Epoch 005/050 | Loss 0.0707 | Win/lose count 5.0/6.0 (-1.0)
Epoch 006/050 | Loss 0.0921 | Win/lose count 5

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [40]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dropout

class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        
        model = Sequential()
        model.add(Conv2D(64, (3, 3),activation='relu',input_shape=(5,5,self.n_state)))
        model.add(Conv2D(32, (3, 3),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4, activation='relu'))
        
        #adam=Adam(lr=lr, decay = 0.9)
        #mse = 'mean_squared_error'
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()

In [18]:
epochs_train = 50

env = Environment(grid_size=size, max_time=T, temperature=0.35)
agent = DQN_CNN(size, lr=.0005, epsilon = 0.1, memory_size=2500, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train') ## 50 epochs
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 3, 3, 64)          1216      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 1, 32)          18464     
_________________________________________________________________
flatten_4 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 4)                 132       
Total params: 19,812
Trainable params: 19,812
Non-trainable params: 0
_________________________________________________________________
Epoch 000/050 | Loss 0.0026 | Win/lose count 7.0/9.0 (-2.0)
Epoch 001/050 | Loss 0.0118 | Win/lose count 2.5/8.0 (-5.5)
Epoch 002/050 | Loss 0.0137 | Win/lose count 4.0/7.0 (-3.0)
Epoch 003/050 | Loss 0.0178 | Win/lose count 7.0/6.0 (1.0)
Epoch 004/050 | Loss 0.0170 | W

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

__Issues:__ the rat can get stuck in a sequence of visited cells. it corresponds to the situation where it sees malicious cells which "scare him" and make him take his way back. Different architectures have been tried (with one or more hidden layers) but the problem always arises. Of course, the parameters can be changed in order to make it less obvious (for example control the exploration part and force the rat to explore more), but overall, this suggests that I need to incorporate a smarter methodology to trade exploration and exploitation off and to deal with visited cells.

__Performance:__ Over the tests, CNN generally outperforms FCN. But I saw cases in which FCN is better. We can understand this as CNN can adapt to more complex functions but can easily overfit the training set. I also noticed that regarding the issue of "getting stuck in visited cells" the cnn is more likely to have this than the FC.

__Temperature effect:__ We observe that the reward increases with temperature. That is because more cheese is added (relative to poison added).

In [22]:
env = Environment(grid_size=size, max_time=T,temperature=0.35)
agent_cnn = DQN_CNN(size, lr=.0005, epsilon = 0.1, memory_size=2500, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.0005, epsilon = 0.1, memory_size=2500, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 3, 3, 64)          1216      
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 1, 1, 32)          18464     
_________________________________________________________________
flatten_7 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 4)                 132       
Total params: 19,812
Trainable params: 19,812
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_

In [23]:
epochs_test = 50

print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
HTML(display_videos('cnn_test10.mp4'))

Test of the CNN
Win/lose count 1.5/0. Average score (1.5)
Win/lose count 0.5/0. Average score (1.0)
Win/lose count 0/0. Average score (0.6666666666666666)
Win/lose count 0.5/0. Average score (0.625)
Win/lose count 1.0/0. Average score (0.7)
Win/lose count 1.5/1.0. Average score (0.6666666666666666)
Win/lose count 0/0. Average score (0.5714285714285714)
Win/lose count 0/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/0. Average score (0.45)
Win/lose count 1.0/0. Average score (0.5)
Win/lose count 1.0/0. Average score (0.5416666666666666)
Win/lose count 0.5/0. Average score (0.5384615384615384)
Win/lose count 3.5/0. Average score (0.75)
Win/lose count 0.5/1.0. Average score (0.6666666666666666)
Win/lose count 0.5/0. Average score (0.65625)
Win/lose count 0.5/0. Average score (0.6470588235294118)
Win/lose count 0/0. Average score (0.6111111111111112)
Win/lose count 0/1.0. Average score (0.5263157894736842)
Win/lose count 1.5/0. Average score (0.575)
Win/l

In [24]:
epochs_test = 50

print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')
HTML(display_videos('fc_test10.mp4'))

Test of the FC
Win/lose count 2.0/0. Average score (2.0)
Win/lose count 1.0/0. Average score (1.5)
Win/lose count 2.5/0. Average score (1.8333333333333333)
Win/lose count 1.0/2.0. Average score (1.125)
Win/lose count 3.5/0. Average score (1.6)
Win/lose count 0.5/2.0. Average score (1.0833333333333333)
Win/lose count 1.0/4.0. Average score (0.5)
Win/lose count 4.0/4.0. Average score (0.4375)
Win/lose count 0.5/2.0. Average score (0.2222222222222222)
Win/lose count 2.0/2.0. Average score (0.2)
Win/lose count 0.5/0. Average score (0.22727272727272727)
Win/lose count 1.0/0. Average score (0.2916666666666667)
Win/lose count 0.5/0. Average score (0.3076923076923077)
Win/lose count 1.5/4.0. Average score (0.10714285714285714)
Win/lose count 0.5/0. Average score (0.13333333333333333)
Win/lose count 0.5/0. Average score (0.15625)
Win/lose count 0/1.0. Average score (0.08823529411764706)
Win/lose count 3.0/2.0. Average score (0.1388888888888889)
Win/lose count 0.5/0. Average score (0.15789473684

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



D'une part, j'ai rajouté une epsilon-exploration aléatoire.
D'autre part, j'ai pénalisé les états déjà visités. J'ai fait une première tentative ou j'ai juste rajouté une penalité sur le board d'un état dès qu'il est visité. Le problème avec ça c'est quand le rat est bloqué dans un coin, il ne fait que perdre dans le reward et augmenter la pénalité (parce qu'elle fait de façon arithmétique).
Du coup, la deuxième facon de faire c'etait d'enregistrer les états visités dans une autre liste

In [41]:
class EnvironmentExploring(object):

    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        
        self.x = 0
        self.y = 1
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        
        self.malus_position[self.x, self.y] = visit_penalty
        
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        ### 3 grilles
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                        self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    

In [42]:

def train_explore(agent,env,epoch, eps, eps_decay,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        eps = eps*eps_decay
        
        agent.set_epsilon(eps)
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 5 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')


In [44]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(30,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [47]:
# Training
epochs_train = 50
eps_decay=0.9
eps = 0.6
visit_penalty = 0.1
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = eps, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, eps, eps_decay, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0151 | Win/lose count 6.0/25.6000000000001 (-19.6000000000001)
Epoch 001/050 | Loss 0.0213 | Win/lose count 13.5/21.900000000000055 (-8.400000000000055)
Epoch 002/050 | Loss 0.0092 | Win/lose count 8.5/23.600000000000065 (-15.100000000000065)
Epoch 003/050 | Loss 0.0283 | Win/lose count 11.0/25.500000000000075 (-14.500000000000075)
Epoch 004/050 | Loss 1.3654 | Win/lose count 15.0/18.4 (-3.3999999999999986)
Epoch 005/050 | Loss 1.7640 | Win/lose count 8.0/24.400000000000063 (-16.400000000000063)
Epoch 006/050 | Loss 1.1428 | Win/lose count 4.5/23.50000000000005 (-19.00000000000005)
Epoch 007/050 | Loss 1.3244 | Win/lose count 17.5/19.100000000000005 (-1.600000000000005)
Epoch 008/050 | Loss 1.3181 | Win/lose count 13.0/19.800000000000008 (-6.800000000000008)
Epoch 009/050 | Loss 1.4110 | Win/lose count 12.0/23.200000000000045 (-11.200000000000045)
Epoch 010/050 | Loss 1.4394 | Win/lose count 18.0/16.899999999999977 (1.1000000000000227)
Epoch 011/050 | Loss 1.1255

In [48]:
# Evaluation
epochs_test = 50
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 23.5/0. Average score (23.5)
Win/lose count 11.5/1.0. Average score (17.0)
Win/lose count 18.5/2.0. Average score (16.833333333333332)
Win/lose count 13.5/4.0. Average score (15.0)
Win/lose count 21.0/0. Average score (16.2)
Win/lose count 17.5/0. Average score (16.416666666666668)
Win/lose count 25.5/2.0. Average score (17.428571428571427)
Win/lose count 12.5/0. Average score (16.8125)
Win/lose count 29.5/3.0. Average score (17.88888888888889)
Win/lose count 25.0/1.0. Average score (18.5)
Win/lose count 11.0/3.0. Average score (17.545454545454547)
Win/lose count 6.0/0. Average score (16.583333333333332)
Win/lose count 6.0/0. Average score (15.76923076923077)
Win/lose count 22.5/0. Average score (16.25)
Win/lose count 9.5/0. Average score (15.8)
Win/lose count 10.5/0. Average score (15.46875)
Win/lose count 27.0/1.0. Average score (16.08823529411765)
Win/lose count 24.0/1.0. Average score (16.47222222222222)
Win/lose count 19.0/2.0. Average score (16.5)
Win/lose count 10

__Remarques__ 
Cette aproche nous montre a quel point le tradoff entre exploration et exploitation est important. Déjà, nous avons vu des méthodes qui ne marche pas. Par exemple, faire que de l'exploitation ou rajouter des pénalités sur le board qui contient les morceaux de fromages. Une autre méthode qui marche pas c'est quand la pénalite des visites est trop grande. Le rats se trouve contraints à faire des mauvais choix, c'est aussi le cas quand la pénalité est trop petite (elle ne sera pas ressentie). Le choix de la meilleure combinaison de parametres est donc tres critique 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***